# Single-column model (SCM) perturbed-physics ensemble (PPE) executor

In [ ]:
!pip install SobolSequence==0.2 > /dev/null

import pandas as pd

from itertools import product
from multiprocessing import Pool
from os.path import exists
from shutil import rmtree
from sobol import sample as sobol
from subprocess import call
from tqdm.notebook import tqdm
from xarray import open_mfdataset


CESM_ROOT = "/opt/ncar/cesm2"

SCRIPT_DIR = f"{CESM_ROOT}/cime/scripts"
MODS_DIR = f"{CESM_ROOT}/components/cam/cime_config/usermods_dirs"

CASE_ROOT = "/tmp/cases"
ARCHIVE_ROOT = "/home/user/archive"

IOP_CASE_DIR = f"{CASE_ROOT}/scm_ppe.base"

In [ ]:
# Intensive observation periods (IOPs).
# https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2018MS001578
iops = [ "arm97"      # ARM Southern Great Plains         Land convection
       , "cgilsS6"    # CFMIP‐GASS SCM/LES Intercomp.     Shallow cumulus
       , "cgilsS11"   # ------------- " -------------     Stratocumulus
       , "cgilsS12"   # ------------- " -------------     Stratus
       , "mpace"      # Mixed Phase Arctic Clouds Exp.    Arctic
       , "sparticus"  # Small Particles in Cirrus         Cirrus, convection
       , "twp06"      # Tropical W. Pacific Convection    Tropical convection
       ]

# Parameter space.
# https://arxiv.org/abs/1711.03675
param_space = { "clubb_gamma_coef" : ( 0.25 ,  0.36 )
              , "clubb_beta"       : ( 1.2  ,  2.6  )
              , "clubb_C8"         : ( 1.5  ,  6.   )
              , "clubb_C11"        : ( 0.2  ,  0.8  )
              , "clubb_C11b"       : ( 0.2  ,  0.8  )
             #, "clubb_penguin"    : ( 0.69 ,  4.20 )  # kek
              }

# Number of quasirandom samples per IOP.
n_samples = 128

# Number of parallel jobs.
n_jobs = 8

In [ ]:
def quasirandom_sample(iops, space, n):
    sample = sobol(len(space), n)

    for column, bounds in zip(sample.T, space.values()):
        low, high = bounds
        column *= high - low
        column += low

    return [dict(zip(space.keys(), x)) for x in sample]


def plan_cases(iops, param_space, n_cases):
    cases = product(quasirandom_sample(iops, param_space, n_cases), iops)

    df = pd.DataFrame([{"iop": iop, **params} for params, iop in cases])
    df.insert(0, "name", [f"scm_ppe.{str(x).rjust(4, '0')}"
                          for x in df.index])

    return df


def run_case(config):
    name, iop = config["name"], config["iop"]
    del config["name"], config["iop"]

    case_dir = f"{CASE_ROOT}/{name}"

    if not exists(f"{ARCHIVE_ROOT}/{name}"):
        rmtree(case_dir, ignore_errors=True)

        assert call([f"{SCRIPT_DIR}/create_clone",
                     "--clone", IOP_CASE_DIR,
                     "--user-mods-dir", f"{MODS_DIR}/scam_{iop}",
                     "--keepexe",
                     "--cime-output-root", case_dir,
                     "--case", case_dir]) == 0

        with open(f"{case_dir}/user_nl_cam", "a") as f:
            print("nhtfrq = -24", file=f)

            for k, v in dict(config).items():
                print(f"{k} = {v}", file=f)

        assert call("./case.submit", cwd=case_dir) == 0

        rmtree(case_dir, ignore_errors=True)


def run_cases(df, n_jobs=None):
    configs = [dict(x[1]) for x in df.iterrows()]

    if not exists(f"{IOP_CASE_DIR}/bld/cesm.exe"):
        rmtree(IOP_CASE_DIR, ignore_errors=True)

        assert call([f"{SCRIPT_DIR}/create_newcase",
                      "--compset", "FSCAM",
                      "--res", "T42_T42",
                      "--user-mods-dir", f"{MODS_DIR}/scam_mandatory",
                      "--case", IOP_CASE_DIR]) == 0

        assert call("./case.setup", cwd=IOP_CASE_DIR) == 0
        assert call("./case.build", cwd=IOP_CASE_DIR) == 0

    with Pool(n_jobs) as p:
        for _ in tqdm(p.imap_unordered(run_case, configs), total=len(configs),
                      mininterval=0., miniters=1):
            pass

In [ ]:
cases = plan_cases(iops, param_space, n_samples)
display(cases)

run_cases(cases, n_jobs)

cases.to_csv("cases.csv", index=False)